In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import json_normalize
import ast

In [2]:
movies_df = pd.read_csv("ETL_movies_main.csv")

In [4]:
movies_df.shape

(45346, 20)

In [8]:
df_languages = movies_df.loc[:,['Movie_id',"spoken_languages"]]
df_languages

,Movie_id,spoken_languages
0,862,"[{'iso_639_1': 'en', 'name': 'English'}]"
1,8844,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso..."
2,15602,"[{'iso_639_1': 'en', 'name': 'English'}]"
3,31357,"[{'iso_639_1': 'en', 'name': 'English'}]"
4,11862,"[{'iso_639_1': 'en', 'name': 'English'}]"
...,...,...
45341,30840,"[{'iso_639_1': 'en', 'name': 'English'}]"
45342,111109,"[{'iso_639_1': 'tl', 'name': ''}]"
45343,67758,"[{'iso_639_1': 'en', 'name': 'English'}]"
45344,227506,[]


In [9]:
# Transform NaN to empty lists
df_languages['spoken_languages'] = df_languages['spoken_languages'].fillna('[]')

In [10]:
# str to dict lists

df_languages['spoken_languages'] = df_languages['spoken_languages'].apply(ast.literal_eval)

In [11]:
# exploding columns
df_exploded = df_languages.explode('spoken_languages', ignore_index=True)
df_exploded

,Movie_id,spoken_languages
0,862,"{'iso_639_1': 'en', 'name': 'English'}"
1,8844,"{'iso_639_1': 'en', 'name': 'English'}"
2,8844,"{'iso_639_1': 'fr', 'name': 'Français'}"
3,15602,"{'iso_639_1': 'en', 'name': 'English'}"
4,31357,"{'iso_639_1': 'en', 'name': 'English'}"
...,...,...
57001,30840,"{'iso_639_1': 'en', 'name': 'English'}"
57002,111109,"{'iso_639_1': 'tl', 'name': ''}"
57003,67758,"{'iso_639_1': 'en', 'name': 'English'}"
57004,227506,NaN


In [12]:
# Separating columns
df_exploded['iso_639_1'] = df_exploded['spoken_languages'].apply(lambda x: x.get('iso_639_1', '') if isinstance(x, dict) else '')
df_exploded['name'] = df_exploded['spoken_languages'].apply(lambda x: x.get('name', '') if isinstance(x, dict) else '')

In [13]:
#joining with original df
df_languages = df_languages.join(df_exploded[['iso_639_1', 'name']])

In [14]:
df_languages.drop(["spoken_languages"],axis=1,inplace=True)

In [15]:
# renaming columns
df_languages.rename(columns={"iso_639_1":"short_language","name":"language"},inplace=True)

In [16]:
df_languages

,Movie_id,short_language,language
0,862,en,English
1,8844,en,English
2,15602,fr,Français
3,31357,en,English
4,11862,en,English
...,...,...,...
45341,30840,en,English
45342,111109,en,English
45343,67758,en,English
45344,227506,cn,广州话 / 廣州話


In [17]:
# to csv

In [18]:
df_languages.to_csv("ETL_movies_languages.csv",index=False)